## 최적화

- 고정형 학습률의 문제점을 지적 -> 적응형 학습률
- 적응형 학습률
    - 1st 모멘텀: 여전히 고정된 학습률
    - 2nd 학습 스케줄링: 학습 후반으로 갈수록 학습률 작게
    - 3rd 아다 그래드: 가중치가 갱신될 때마다 학습률 작게 -> 0으로 수렴될 수도..
    - 4th 아담 옵티마이저: 기존 + 모멘텀

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

#### 데이터 전처리

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

# 캘리포니아 주택 정보
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Target"] = california.target
df.tail()

# 데이터 정규화
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1])
df.tail()

#### 모델 생성

In [ ]:
# 입출력 데이터 구성
data = torch.from_numpy(df.values).float()
x = data[:, :-1]
y = data[:, -1:]
print(x.size(), y.size())

# 모델 생성 및 옵티파이저 설정
model = nn.Sequential(
    nn.Linear(x.size(-1), 8),
    nn.LeakyReLU(),
    nn.Linear(8, 7),
    nn.LeakyReLU(),
    nn.Linear(7, 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 3),
    nn.LeakyReLU(),
    nn.Linear(3, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1)),
)

#### 학습

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

# 학습에 필요한 세팅값 설정
n_epochs = 4000
batch_size = 256
print_interval = 200
learning_rate = 1e-2

for i in range(n_epochs):
    # 임의의 순서로 미니배치로 나누기
    indices = torch.randperm(x.size(0))
    x_ = torch.index_select(x, dim=0, index=indices)
    y_ = torch.index_select(y, dim=0, index=indices)
    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)

    y_hat = []
    total_loss = 0
    
    # 경사하강법
    for x_i, y_i in zip(x_, y_):
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  

        total_loss += float(loss)
        y_hat += [y_hat_i]

    # 손실값
    total_loss = total_loss / len(x_)        
    if (i + 1) % print_interval == 0:
        print('Epoch %d: train loss=%.4e' % (i + 1, total_loss))

#### 평가

In [ ]:
y_hat = torch.cat(y_hat, dim=0)
y = torch.cat(y_, dim=0)

df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach().numpy(),
                  columns=["y", "y_hat"])

sns.pairplot(df, height=5)
plt.show()